# Hidden Markov Models For Word Tagging
My attempt to build a trigrame model by extending the bigram model built by Katrin Erk which is found here:

http://www.katrinerk.com/courses/python-worksheets/hidden-markov-models-for-pos-tagging-in-python

## Introduction
When we speak, because we took english in school, we know that each word we say has a part of speech associated with that word. For example if we consider the sentence "Abraham ran swiftly" we know that "Abraham" is a noun, "ran" is a verb and "swiftly" is an adjective. Transfering this knowledge to a computer to do natural language processing can be tricky since we have words that can have an ambiguous part of speech. One example is the word "lead."  It can mean the metal, in which case it is a noun, but it can also be a verb as as with the sentence "I will lead us to victory!" So we need context to clarify this potential ambiguity. One way we can incoprperate context is by using a Markov chain which can models sequential processes. The only problem left is that the computer never actually "sees" the tags. When we write sentences we never include where the word is a noun or verb, etc. The Computer only sees the words. So to model this, we extend the idea of a Markov chain to a _hidden Markov chain_. The idea is that we know there is a Markov process that is hidden, in this case the word tags, but the information we do see, i.e. the words themselves, give us enough information to make inferences about the underlying hidden process.

To be more specific, a Markov Chain is a process in which the probability of moving to the next state is only dependent on the current state and perhaps a few of the previous states, but the distant past. So, for a sequence $t_0, \ldots, t_{k-3}, t_{k-2}, t_{k-1}, t_{k}$ The probability of $t_k$ given the entire history is given by

\begin{align*}
  P( t_{k+1} | t_0, \ldots, t_{k-3}, t_{k-2}, t_{k-1}, t_{k}) 
   &= P( t_{k+1} | t_{k}) &&\text{First Order Markov Chain}\\
  P( t_{k+1} | t_0, \ldots, t_{k-3}, t_{k-2}, t_{k-1}, t_{k}) 
   &= P( t_{k+1} |  t_{k-1}, t_{k}) &&\text{Second Order Markov Chain}\\
  P( t_{k+1} | t_0, \ldots, t_{k-3}, t_{k-2}, t_{k-1}, t_{k}) 
   &= P( t_{k+1} |  t_{k-2}, t_{k-1}, t_{k}) &&\text{Third Order Markov Chain}\\
\end{align*}

In other words, a first order Markov chain doesn't need any past information except for the last state to calculate the probability of the next state. A second order Markov chain only needs the last two states to calculate the probability, and so on. 

In this example, we will be modeling the tags with a second order Markov chain. The $\{t_k\}_{k\in\mathbb{N}}$ sequence will be our tags since we know that if we start a sentence with a determiner followed by a noun, we know there it is very likely that a verb will come next. At the same time the distant past is irrelevant because the next tag in a sentence will not depend on the any tags that apeared in the previous paragraph. We can model this pictorally as shown below. 
<img src="TagMC.png" style="width:500px">
The function $P(T_{k}|T_{k-2}, T_{k-1})$ is the probability of transitioning to tag $T_k$ given that the previous two tags were $T_{k-2}$ and $T_{k-1}$.

But again, the computer never sees the tags, it only sees the words. We can include the words in the diagram as shown below.
<img src="HMM.png" style="width:500px">
And the function $e(W|T)$ is the _emission_ probability of seeign word $W$ given that the current tag is $T$. With this model, we can calculate the joint probability of all words and tags as:

$$   
  P(w_1, \dots, w_K, t_1, \dots, t_K) 
   = P(t_1)\cdot P(t_2|t_1)\cdot \prod_{i=3}^{K}P(t_{i}|t_{i-2}, t_{i-1})\cdot \prod_{j=1}^{K}P(w_{j}|t_{j})
$$

With the joint probability distribution in hand, it is easy to find conditional distributions. We specifically want $P(t_1, \dots, t_K|w_1, \dots, w_K)$ because we can choose the best tag sequence $\{t_k^*\}_{k=1}^{K}$ to be the sequnce that maximizes the conditioninal distribution probability. 

\begin{align*}
  \{t_k^*\}_{k=1}^{K} = \mathop{\text{argmax}}_{t_1,\dots,t_K} P(t_1, \dots, t_K|w_1, \dots, w_K)
\end{align*}

We will use the following lbraries for this task.

In [12]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import brown

The brown corpus contain about a million tagged words you can see descriptioins of the tags using `nltk.help.brown_tagset()`. To easy make things a bit easier, I'll convert all words to lowercase. This mean that "Cow" and "cow" are treated as the same word instead of different words which results in a smaller emission matrix. I'll also only use the base form of the tag, which means that `DO+PPSS` and `DO*` are both converted to `DO`. This has the effect if decreasing the transition matrix and the emission matrix. First let me define a function to pull the base tag.

In [13]:
def getBaseTag(tagString):
    """Gets a simple version of an input tag. 
    
    Example:
    >>> getTag("WDT+BER+PP")
    [1] 'WDT'
    >>> getTag("--")
    [2] '--'
      
    Input:
      tagString (str): The full tag

    Output:
      str: if `tagString` is a word tag, it returns the base of the tag
        if `tagString` is a punctuation tag, it returns `tagString`
    """
    reTag = re.search("[A-Z]+", tagString, flags=0)
    if reTag:
        return reTag.group(0)
    else:
        return tagString

In [14]:
brown_tags_words = [ ]
for sent in brown.tagged_sents():
    # sent is a list of word/tag pairs
    # add START/START at the beginning
    brown_tags_words.append( ("START", "START") )
    brown_tags_words.append( ("START", "START") )
    # then all the tag/word pairs for the word/tag pairs in the sentence.
    # shorten tags to 2 characters each
    brown_tags_words.extend([ (getBaseTag(tag), word) for (word, tag) in sent ])
    # then END/END
    brown_tags_words.append( ("END", "END") )    
    brown_tags_words.append( ("END", "END") )
    
# Placing data into data frame for convinience
brown_df = pd.DataFrame({"words":list(map(lambda x: x[1], brown_tags_words)),
                         "tags" :list(map(lambda x: x[0], brown_tags_words))})

del brown_tags_words, brown # no longer needed

At this points, I put all of the words and tags together in one data frame as opposed to the way it was before, which was split up into sentences. So now the data looks like this:

In [15]:
brown_df.head(10)

,tags,words
0,START,START
1,START,START
2,AT,The
3,NP,Fulton
4,NN,County
5,JJ,Grand
6,NN,Jury
7,VBD,said
8,NR,Friday
9,AT,an


But since I want to build a trigram model, that means I need to look a the previous two states. To acomplish this, I will pair up the words into pairs sperarated by the character `->`. The inital state will have the variable `initalState` which will be the previous two tags and the current state will have the variable name of `currentState` and will represent the tag of the previous word followed by the tag for the current word. 

To accomplish this, I will join the table with itself, but with the index decremented. And I'lld to this twice so that I have the three most recent tags. I will also add a column called `count` to count the occurances of each state.

In [16]:
trigram_df = brown_df[["tags"]].join(brown_df[["tags"]].shift(-1), lsuffix="_2") \
                .join(brown_df[["tags","words"]].shift(-2), lsuffix="_1", rsuffix="_0") \
                .dropna()

# This will be used to count occurances of each state 
trigram_df["counts"] = np.ones(len(trigram_df))

# join two concurent tags that define the states.
trigram_df["initialState"] = trigram_df.tags_2 + " -> " + trigram_df.tags_1
trigram_df.drop('tags_2', axis=1, inplace=True)
trigram_df["currentState"] = trigram_df.tags_1 + " -> " + trigram_df.tags_0
trigram_df.drop(['tags_1'], axis=1, inplace=True)
trigram_df.rename(columns={"tags_0":"currentTag"}, inplace=True)

So far the data looks like this:

In [17]:
trigram_df.head(5)

,currentTag,words,counts,initialState,currentState
0,AT,The,1.0,START -> START,START -> AT
1,NP,Fulton,1.0,START -> AT,AT -> NP
2,NN,County,1.0,AT -> NP,NP -> NN
3,JJ,Grand,1.0,NP -> NN,NN -> JJ
4,NN,Jury,1.0,NN -> JJ,JJ -> NN


## Calculating The Transistion Matrix

With the states defined, I need to count the number of transitions from `initialSate` to `currentState` followed by normalizing by dividing by the sum so each row represents a probability distribution. 

In [18]:
transitionDF = trigram_df[["initialState", "currentState", "counts"]] \
    .pivot_table(columns="currentState", 
                       index="initialState", 
                       values="counts", 
                       fill_value=0,
                       aggfunc=np.sum) \
    .apply(lambda row: row/row.sum(), axis=1)

Now we have the transition matrix in the form of a dataframe. Which is a large sparce matrix that is too ugly to show, but if I want to find what percentage of times the sequence `NN`, `VBD` was preceded by `NR`, I just look for the initial state of `'NN -> VBD'` and current state of `'VBD -> NR'` like so:

In [19]:
transitionDF["VBD -> NR"]["NN -> VBD"]

0.0045772751750134625

Which means that from the state `'NN-TL -> VBD'`, the next state was `'VBD -> NR'` about 0.46% of the time. Mathematically, this is the same as ${\mathbb{P}(t_k={\tt NR} \,|\, t_{k-2}={\tt NN}, t_{k-1}={\tt VBD})}$

## Calculating The Emission Matrix
The emission matrix will the contain the probabilities of having a certain work given the current state, which recall is the previous tag followed by the current tag. 

In [20]:
emissionDF = trigram_df[["words", "currentTag", "counts"]] \
    .pivot_table(columns="words", 
                       index="currentTag", 
                       values="counts", 
                       fill_value=0,
                       aggfunc=np.sum) \
    .apply(lambda row: row/row.sum(), axis=1) 

In [21]:
emissionDF["the"]["AT"]

0.63294205516921187

This means that given that the current tag is in the state `"AT"`, the probability of getting the word `"the"` is about 63%. This is equivalent to $\mathbb{P}(w_k={\tt The} \;|\; t_k = {\tt AT})$.